In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder

import hashlib


In [12]:
data = pd.read_csv('./Doceree-HCP_Train.csv',encoding='latin1')
testdata = pd.read_csv('./Doceree-HCP_Test.csv',encoding='latin1')


In [13]:
data.drop(['ID', 'CHANNELTYPE','USERAGENT'], axis=1, inplace=True)
testdata.drop(['ID', 'CHANNELTYPE','USERAGENT'], axis=1, inplace=True)

In [14]:
label_encoder = LabelEncoder()

# Fit and transform the 'DEVICETYPE' column
data['DEVICETYPE'] = label_encoder.fit_transform(data['DEVICETYPE'])
data['USERCITY'] = label_encoder.fit_transform(data['USERCITY'])
data['PLATFORMTYPE'] = label_encoder.fit_transform(data['PLATFORMTYPE'])
data['TAXONOMY'] = label_encoder.fit_transform(data['TAXONOMY'])
data['URL'] = label_encoder.fit_transform(data['URL'])
data['KEYWORDS'] = label_encoder.fit_transform(data['KEYWORDS'])
data['BIDREQUESTIP'] = label_encoder.fit_transform(data['BIDREQUESTIP'])
data['USERPLATFORMUID'] = label_encoder.fit_transform(data['USERPLATFORMUID'])

data['USERZIPCODE'].fillna(-1, inplace=True)
data['IS_HCP'].fillna(0, inplace=True)




# Fit and transform the 'DEVICETYPE' column
testdata['DEVICETYPE'] = label_encoder.fit_transform(testdata['DEVICETYPE'])
testdata['USERCITY'] = label_encoder.fit_transform(testdata['USERCITY'])
testdata['PLATFORMTYPE'] = label_encoder.fit_transform(testdata['PLATFORMTYPE'])
# testdata['TAXONOMY'] = label_encoder.fit_transform(testdata['TAXONOMY'])
testdata['URL'] = label_encoder.fit_transform(testdata['URL'])
testdata['KEYWORDS'] = label_encoder.fit_transform(testdata['KEYWORDS'])
testdata['BIDREQUESTIP'] = label_encoder.fit_transform(testdata['BIDREQUESTIP'])
testdata['USERPLATFORMUID'] = label_encoder.fit_transform(testdata['USERPLATFORMUID'])

testdata['USERZIPCODE'].fillna(-1, inplace=True)
# testdata['IS_HCP'].fillna(0, inplace=True)



In [12]:
import time
data.to_csv(f'data{time.time()}.csv', index=False)
testdata.to_csv(f'testdata{time.time()}.csv', index=False)


In [15]:
data

,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,PLATFORMTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,0,2,6580,22183,3177,97206.0,3,666,459,207,0.0
1,0,2,19253,40576,130,22202.0,3,708,109,207,0.0
2,0,2,19704,34968,2701,83654.0,3,666,459,207,0.0
3,0,3,4193,14676,4420,229114624.0,3,327,274,149,1.0
4,1,7,10056,33844,1767,77008.0,3,1631,258,207,0.0
...,...,...,...,...,...,...,...,...,...,...,...
113932,0,2,22037,1063,3067,19120.0,3,819,149,69,1.0
113933,0,2,435,10705,4043,914014213.0,3,852,68,149,1.0
113934,0,7,10581,32808,4420,98105.0,3,1576,922,112,1.0
113935,1,2,22764,36868,4322,19808.0,3,2387,1680,149,1.0


In [16]:
features = ['DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP','USERPLATFORMUID', 'USERCITY', 'USERZIPCODE', 'PLATFORMTYPE', 'URL','KEYWORDS']

X = data[features]
y = data['IS_HCP']


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, y_train = X,y

In [18]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Create the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Best Model Accuracy: {accuracy:.4f}")

In [8]:
print(best_params);
print(best_model);
print(accuracy);

{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}
RandomForestClassifier(n_estimators=300, random_state=42)
0.99951728980165


In [9]:
data = pd.read_csv('./Doceree-HCP_Test.csv',encoding='latin1')

columns = [ 'DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP', 'USERPLATFORMUID',
           'USERCITY', 'USERZIPCODE', 'USERAGENT', 'PLATFORMTYPE', 'CHANNELTYPE', 'URL', 'KEYWORDS']


predTest = best_model.predict(testdata);

data.drop(columns, axis=1, inplace=True)

testdata["IS_HCP"] = predTest

testdata.to_csv('outputFile.csv', index=False)


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- TAXONOMY
